# Metric Learning

In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from omegaconf import OmegaConf
from model.model import KLLossMetricLearning, loss_different_class, loss_same_class, KL_d, KL_dreg
from data_utils import cub2011
from data_utils import stanford_cars

## Download data

In [2]:
train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

In [3]:
cars_trainset = stanford_cars.download_cars('./data', train=True, download=True, transforms=train_transform)
# cars_testset = ImageFolder(DATA_DIR_TEST, transform = test_tfms)

cub_trainset = cub2011.Cub2011('./data/cub2011', train=True, download=True)
cub_testset = cub2011.Cub2011('./data/cub2011', train=False, download=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading stanford-car-dataset-by-classes-folder.zip to ./data/stanford-car-dataset-by-classes-folder


100%|██████████| 1.83G/1.83G [01:47<00:00, 18.2MB/s]



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
cars_trainloader = torch.utils.data.DataLoader(cars_trainset, batch_size=batch_size,
                                          shuffle=True)
cars_testloader = torch.utils.data.DataLoader(cars_testset, batch_size=512,
                                         shuffle=False)

In [5]:
conf_path = os.path.join("configs", "model.yaml")
conf = OmegaConf.load(conf_path)
model = KLLossMetricLearning(**conf.get("model"))

In [25]:
images = torch.ones(64, 3, 32, 32)
model(images)
m1 = torch.tensor([[0.1, 0.3, 0.6], [0.5, 0, 0.5]])
m2 = torch.tensor([[0.1, 0.3, 0.6], [0.5, 0, 0.5]])
s1 = torch.tensor([[0.1, 0.3, 0.6], [0.5, 0, 0.5]])
s2 = torch.tensor([[0.1, 0.3, 0.6], [0.5, 0, 0.5]])
print(loss_same_class((m1, s1), (m2, s2), 0.5, 1))
print(loss_different_class((m1, s1), (m2, s2), 0.5, 1))

tensor(-3.2365)
tensor(6.1427)


In [24]:
0.5 * KL_dreg((m1.log(), s1), (m2.log(), s2))

tensor([6.7127,    inf])

In [12]:
torch.nn.KLDivLoss(size_average=False)(m1.log(), s1)

/home/richard/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


tensor(0.)